# 数据处理

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("../train_set.csv")
test = pd.read_csv("../test_set.csv")

In [3]:
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [4]:
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [5]:
train[train.contact == "unknown"].count()

ID           7281
age          7281
job          7281
marital      7281
education    7281
default      7281
balance      7281
housing      7281
loan         7281
contact      7281
day          7281
month        7281
duration     7281
campaign     7281
pdays        7281
previous     7281
poutcome     7281
y            7281
dtype: int64

In [6]:
def virtualizationData(items):
    '''
    将数据转成数字
    '''
    item_set = set(items)
    item_ids = {}
    new_item = []
    for (i, item) in enumerate(item_set):
        item_ids[item] = i
    for item in items:
        new_item.append(item_ids[item])
    return new_item
itrain = pd.DataFrame()
itest = pd.DataFrame()
code_cols = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]
for col in code_cols:
    itrain[col] = virtualizationData(train[col])
    itest[col] = virtualizationData(test[col])
ori_cols = ["age", "balance", "day", "duration", "campaign", "pdays", "previous"]
for col in ori_cols:
    itrain[col] = train[col]
    itest[col] = test[col]
itrain["y"] = train["y"]
itrain.head()


,job,marital,education,default,housing,loan,contact,month,poutcome,age,balance,day,duration,campaign,pdays,previous,y
0,11,2,3,1,0,1,0,9,1,43,291,9,150,2,-1,0,0
1,8,1,2,1,0,1,2,1,2,42,5076,7,99,1,251,2,0
2,3,2,1,1,0,0,2,3,1,47,104,14,77,2,-1,0,0
3,11,0,1,1,0,0,2,3,1,28,-994,18,174,2,-1,0,0
4,8,1,1,1,0,1,0,9,1,42,2974,21,187,5,-1,0,0


In [7]:
import seaborn as sns
cm = train.corr()
sns.set(font_scale=1.5)
hm = sns.heatmap(cm, cbar=True, annot=True,
                 square=True,fmt=".2f",annot_kws={"size":15},yticklabels=train.columns,xticklabels=train.columns)
                 #square=True,fmt=".2f",annot_kws={"size":15},yticklabels=train.columns,xticklabels=train.columns)
#sns.pairplot(itrain, size=1.5)

In [8]:
itrain.groupby("y").count()

,job,marital,education,default,housing,loan,contact,month,poutcome,age,balance,day,duration,campaign,pdays,previous
y,,,,,,,,,,,,,,,,
0,22356,22356,22356,22356,22356,22356,22356,22356,22356,22356,22356,22356,22356,22356,22356,22356
1,2961,2961,2961,2961,2961,2961,2961,2961,2961,2961,2961,2961,2961,2961,2961,2961


In [9]:
from imblearn.over_sampling import SMOTE  # 过抽样处理库SMOTE
from imblearn.under_sampling import RandomUnderSampler  # 欠抽样处理库RandomUnderSampler
from imblearn.ensemble import EasyEnsemble  # 简单集成方法EasyEnsemble
#修正样本

# 模型

In [87]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, Y_train, Y_test = train_test_split(itrain[code_cols + ori_cols], itrain["y"],
                                                    test_size = 0.3, random_state = 0)
all_X = itrain[code_cols + ori_cols]
all_Y = itrain["y"]

In [11]:
def modelResult(model, name, X_train, X_test, Y_train, Y_test):
    """
    模型拟合预估
    """
    model.fit(X_train, Y_train)
    train_predict = model.predict_proba(X_train)
    test_predict = model.predict_proba(X_test)
    print("model:%s train AUC[%.4f] test AUC[%.4f] test ACC[%.4f]" % 
          (name, 
           metrics.roc_auc_score(Y_train, train_predict[:,1]),
           metrics.roc_auc_score(Y_test, test_predict[:,1]),
           metrics.accuracy_score(Y_test, model.predict(X_test))
          ))
    return model

# 模型介绍
## GBDT


## RF
随机森林指的是利用多棵树对样本进行训练并预测的一种分类器。在机器学习中，随机森林是一个包含多个决策树的分类器， 并且其输出的类别是由个别树输出的类别的众数而定。 Leo Breiman和Adele Cutler发展出推论出随机森林的算法。 而 "Random Forests" 是他们的商标。 这个术语是1995年由贝尔实验室的Tin Kam Ho所提出的随机决策森林（random decision forests）而来的。这个方法则是结合 Breimans 的 "Bootstrap aggregating" 想法和 Ho 的"random subspace method"以建造决策树的集合。算法的表现形式[3]：

l    用N来表示训练用例（样本）的个数，M表示特征数目。

l    输入特征数目m，用于确定决策树上一个节点的决策结果；其中m应远小于M。

l    从N个训练用例（样本）中以有放回抽样的方式，取样N次，形成一个训练集（即bootstrap取样），并用未抽到的用例（样本）作预测，评估其误差。

l    对于每一个节点，随机选择m个特征，决策树上每个节点的决定都是基于这些特征确定的。根据这m个特征，计算其最佳的分裂方式。

l    每棵树都会完整成长而不会剪枝，这有可能在建完一棵正常树状分类器后会被采用）。

关于调参，随机森林有几个比较重要的参数[4]：

l  max_features：

随机森林允许单个决策树使用特征的最大数量。 Python为最大特征数提供了多个可选项。 下面是其中的几个：

n  Auto/None ：简单地选取所有特征，每颗树都可以利用他们。这种情况下，每颗树都没有任何的限制。

n  sqrt ：此选项是每颗子树可以利用总特征数的平方根个。 例如，如果变量（特征）的总数是100，所以每颗子树只能取其中的10个。“log2”是另一种相似类型的选项。

n  0.2：此选项允许每个随机森林的子树可以利用变量（特征）数的20％。如果想考察的特征x％的作用， 我们可以使用“0.X”的格式。

n  max_features如何影响性能和速度？ 增加max_features一般能提高模型的性能，因为在每个节点上，我们有更多的选择可以考虑。 然而，这未必完全是对的，因为它降低了单个树的多样性，而这正是随机森林独特的优点。 但是，可以肯定，你通过增加max_features会降低算法的速度。 因此，你需要适当的平衡和选择最佳max_features。

l  n_estimators：

在利用最大投票数或平均值来预测之前，你想要建立子树的数量。 较多的子树可以让模型有更好的性能，但同时让你的代码变慢。 你应该选择尽可能高的值，只要你的处理器能够承受的住，因为这使你的预测更好更稳定。

l  min_sample_leaf：

如果您以前编写过一个决策树，你能体会到最小样本叶片大小的重要性。 叶是决策树的末端节点。 较小的叶子使模型更容易捕捉训练数据中的噪声。 一般来说，我更偏向于将最小叶子节点数目设置为大于50。在你自己的情况中，你应该尽量尝试多种叶子大小种类，以找到最优的那个。


## LR
logistic回归又称logistic回归分析，是一种广义的线性回归分析模型，常用于数据挖掘，疾病自动诊断，经济预测等领域。例如，探讨引发疾病的危险因素，并根据危险因素预测疾病发生的概率等。Logistic回归模型的适用条件：

l  因变量为二分类的分类变量或某事件的发生率，并且是数值型变量。但是需要注意，重复计数现象指标不适用于Logistic回归。

l  残差和因变量都要服从二项分布。二项分布对应的是分类变量，所以不是正态分布，进而不是用最小二乘法，而是最大似然法来解决方程估计和检验问题。

l  自变量和Logistic概率是线性关系

l  各观测对象间相互独立。

 

原理：如果直接将线性回归的模型扣到Logistic回归中，会造成方程二边取值区间不同和普遍的非直线关系。因为Logistic中因变量为二分类变量，某个概率作为方程的因变量估计值取值范围为0-1，但是，方程右边取值范围是无穷大或者无穷小。所以，才引入Logistic回归。

Logistic回归实质：发生概率除以没有发生概率再取对数。就是这个不太繁琐的变换改变了取值区间的矛盾和因变量自变量间的曲线关系。究其原因，是发生和未发生的概率成为了比值 ，这个比值就是一个缓冲，将取值范围扩大，再进行对数变换，整个因变量改变。不仅如此，这种变换往往使得因变量和自变量之间呈线性关系，这是根据大量实践而总结。所以，Logistic回归从根本上解决因变量要不是连续变量怎么办的问题。还有，Logistic应用广泛的原因是许多现实问题跟它的模型吻合。例如一件事情是否发生跟其他数值型自变量的关系。[5]
logistic回归又称logistic回归分析，是一种广义的线性回归分析模型，常用于数据挖掘，疾病自动诊断，经济预测等领域。例如，探讨引发疾病的危险因素，并根据危险因素预测疾病发生的概率等。Logistic回归模型的适用条件：

l  因变量为二分类的分类变量或某事件的发生率，并且是数值型变量。但是需要注意，重复计数现象指标不适用于Logistic回归。

l  残差和因变量都要服从二项分布。二项分布对应的是分类变量，所以不是正态分布，进而不是用最小二乘法，而是最大似然法来解决方程估计和检验问题。

l  自变量和Logistic概率是线性关系

l  各观测对象间相互独立。
l  penalty：惩罚项，str类型，可选参数为l1和l2，默认为l2。用于指定惩罚项中使用的规范。newton-cg、sag和lbfgs求解算法只支持L2规范。
L1G规范假设的是模型的参数满足拉普拉斯分布，L2假设的模型参数满足高斯分布，所谓的范式就是加上对参数的约束，使得模型更不会过拟合(overfit)，
但是如果要说是不是加了约束就会好，这个没有人能回答，只能说，加约束的情况下，理论上应该可以获得泛化能力更强的结果。

l  c：正则化系数λ的倒数，float类型，默认为1.0。必须是正浮点型数。像SVM一样，越小的数值表示越强的正则化。
## SVM
SVM(Support Vector Machine)指的是支持向量机，是常见的一种判别方法。在机器学习领域，是一个有监督的学习模型，通常用来进行模式识别、分类以及回归分析。SVM的主要思想可以概括为两点[1]：

l  它是针对线性可分情况进行分析，对于线性不可分的情况，通过使用非线性映射算法将低维输入空间线性不可分的样本转化为高维特征空间使其线性可分，从而使得高维特征空间采用线性算法对样本的非线性特征进行线性分析成为可能。

l  它基于结构风险最小化理论之上在特征空间中构建最优超平面，使得学习器得到全局最优化，并且在整个样本空间的期望以某个概率满足一定上界。

       在Sklearn中，svm具有分类和回归两种（分别是SVC、SVR），本实验处理的是分类问题，故使用SVC。

       对于调参，SVM模型有两个非常重要的参数C与gamma。其中 C是惩罚系数，即对误差的宽容度。c越高，说明越不能容忍出现误差,容易过拟合。C越小，容易欠拟合。C过大或过小，泛化能力变差gamma是选择RBF函数作为kernel后，该函数自带的一个参数。隐含地决定了数据映射到新的特征空间后的分布，gamma越大，支持向量越少，gamma值越小，支持向量越多[2]。支持向量的个数影响训练与预测的速度。本实验则是简单的调整了C，C一般可以选择为：10^t , t=- 4..4就是0.0001 到10000，选择的越大，表示对错误例惩罚程度越大，可能会导致模型过拟合，代码中默认不进行调参是1.0，调参之后是0.1-100,step是0.1，调参之后，将所有的结果放入LIST中，然后，返回其中最大的评分值，其参数C和模型作为最优参数C和最优模型。
## KNN
   邻近算法，或者说K最近邻(kNN，k-NearestNeighbor)分类算法是数据挖掘分类技术中最简单的方法之一。所谓K最近邻，就是k个最近的邻居的意思，说的是每个样本都可以用它最接近的k个邻居来代表。kNN算法的核心思想是如果一个样本在特征空间中的k个最相邻的样本中的大多数属于某一个类别，则该样本也属于这个类别，并具有这个类别上样本的特性。该方法在确定分类决策上只依据最邻近的一个或者几个样本的类别来决定待分样本所属的类别。 kNN方法在类别决策时，只与极少量的相邻样本有关。由于kNN方法主要靠周围有限的邻近的样本，而不是靠判别类域的方法来确定所属类别的，因此对于类域的交叉或重叠较多的待分样本集来说，kNN方法较其他方法更为适合[7]。

       关于调参[8]：

KNeighborsClassifier方法中含有8个参数（以下前两个常用）：

l  n_neighbors : int, optional (default = 5)：K的取值，默认的邻居数量是5；

l  weights：确定近邻的权重，“uniform”权重一样，“distance”指权重为距离的倒数，默认情况下是权重相等。也可以自己定义函数确定权重的方式；

l  algorithm : {'auto', 'ball_tree', 'kd_tree', 'brute'},optional：计算最近邻的方法，可根据需要自己选择；


## XGBoost调优

In [98]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV 
def modelFit(model, X, Y, cv_folds=5, early_stopping_rounds=100):
    """
    树的个数
    """
    params = model.get_xgb_params()
    xgtrain = xgb.DMatrix(X, label=Y)
    cv_params = xgb.cv(params, xgtrain, num_boost_round=model.get_params()['n_estimators'], nfold=cv_folds,
           metrics='auc', early_stopping_rounds=early_stopping_rounds)
    #print(cv_params)
    model.set_params(n_estimators=cv_params.shape[0])
    print(model)
def modelFitParams(X, Y):
    param_test1 = {
        #step1
        #'max_depth':range(3,10,1),
        #'min_child_weight':range(1,6,1), 
        #step2 0.2
        #'gamma':[i/10.0 for i in range(0,5)],
        #step3 0.8 0.9
        # 'subsample':[i/10.0 for i in range(6,10)],
        # 'colsample_bytree':[i/10.0 for i in range(6,10)]
        #step4 正则化调优
        #'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
        #step5 学习率
        'learning_rate':[i/100.0 for i in range(1, 20)],
    }
    gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate =0.1,
                                                      n_estimators=152,
                                                      max_depth=6,
                                                      min_child_weight=4,
                                                      gamma=0.2,
                                                      subsample=0.9,
                                                      colsample_bytree=0.8,
                                                      objective= 'binary:logistic',
                                                      nthread=4,
                                                      scale_pos_weight=1,
                                                      reg_alpha=1e-5,
                                                      seed=27), 
                            param_grid = param_test1,
                            scoring='roc_auc',
                            n_jobs=4,
                            iid=False, 
                            cv=5)
    gsearch1.fit(X, Y)
    print(gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_)
modelFitParams(X_train, Y_train)
xgb_model = xgb.XGBClassifier(max_depth=6,n_estimators=200, num_round = 5, learning_rate=0.1,subsample=0.7
                             , colsample_bytree=0.7)
#modelFit(models["xgb"], X_train, Y_train)

{'mean_fit_time': array([4.079708  , 3.88230977, 3.96591282, 4.22340617, 4.37811127,
       3.89016228, 4.22154446, 4.56992168, 3.90031977, 3.39647675,
       3.3395956 , 3.31014233, 4.26095185, 3.96596942, 3.63716722,
       3.79310679, 3.58740282, 3.68983827, 3.24412942]), 'std_fit_time': array([0.28542795, 0.38773187, 0.29226028, 0.28203301, 0.1510868 ,
       0.09211098, 0.26319956, 0.0360257 , 0.1818952 , 0.04457788,
       0.03425302, 0.02828785, 0.06625708, 0.4093231 , 0.16265969,
       0.02615024, 0.05709379, 0.03446675, 0.38794779]), 'mean_score_time': array([0.05356555, 0.05116343, 0.05669961, 0.06162376, 0.06912975,
       0.06384468, 0.07689433, 0.09101448, 0.05766506, 0.05596738,
       0.05379577, 0.05572591, 0.05814404, 0.0534626 , 0.06453185,
       0.05961485, 0.05449262, 0.05986052, 0.04982262]), 'std_score_time': array([0.00354647, 0.0043078 , 0.00446319, 0.01281127, 0.0271714 ,
       0.01347556, 0.01168259, 0.0187665 , 0.00449028, 0.00405931,
       0.0029707 , 0.

In [105]:
"""
LR
l  penalty：惩罚项，str类型，可选参数为l1和l2，默认为l2。用于指定惩罚项中使用的规范。newton-cg、sag和lbfgs求解算法只支持L2规范。
L1G规范假设的是模型的参数满足拉普拉斯分布，L2假设的模型参数满足高斯分布，所谓的范式就是加上对参数的约束，使得模型更不会过拟合(overfit)，
但是如果要说是不是加了约束就会好，这个没有人能回答，只能说，加约束的情况下，理论上应该可以获得泛化能力更强的结果。

l  c：正则化系数λ的倒数，float类型，默认为1.0。必须是正浮点型数。像SVM一样，越小的数值表示越强的正则化。
"""
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

#help(xgb.cv)
models = {
#    "gbdt" : GradientBoostingClassifier(random_state=100, learning_rate=0.15),
    #"gbdt2" : GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=5),
#    #10颗树
#    "LR" : LogisticRegression(C=0.1,max_iter=100),
    "xgb" : xgb.XGBClassifier(learning_rate =0.1,
                                                n_estimators=152,
                                                max_depth=6,
                                                min_child_weight=4,
                                                gamma=0.2,
                                                subsample=0.9,
                                                colsample_bytree=0.8,
                                                objective= 'binary:logistic',
                                                nthread=4,
                                                scale_pos_weight=1,
                                                reg_alpha=1e-5,
                                                seed=27),
#    "xgb10" : xgb.XGBClassifier(max_depth=6,n_estimators=100,num_round = 10),
#    "RF" : RandomForestClassifier(n_estimators=100, max_depth=13, max_features="sqrt", min_samples_leaf=10),
#    "RF_gini" : RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini', min_samples_leaf=5),
#    "RF_entropy" : RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='entropy', min_samples_leaf=5),
#    "ET_gini" : ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='gini', min_samples_leaf=5),
#    "ET_entropy" : ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy', min_samples_leaf=5),
#    "RF2" : RandomForestClassifier(n_estimators=100),
#    "LR" : LogisticRegression(C=1.0, penalty="l2"),
#    #"LR-lbfgs" : LogisticRegression(C=1.0, penalty="lbfgs"),
#    "KNN" : KNeighborsClassifier(n_neighbors=40)
    #, "SVM" : SVC(C=10, probability=True)
    
}

#print(xgb.XGBClassifier(max_depth=6,n_estimators=100,num_round = 5))
#clf = modelResult(models["gbdt"], "gbdt", X_train, X_test, Y_train, Y_test)
fit_model = {}
for name in models:
    fit_model[name] = modelResult(models[name], name, X_train, X_test,
                                  Y_train, Y_test)
    #fit_model[name] = modelResult(models[name], name, all_X, X_test,
    #                              all_Y, Y_test)
    print(models[name])
#    fit_model[name] = modelResult(models[name], name, X_train.append(X_train, ignore_index=True), X_test,
#                                  Y_train.append(Y_train, ignore_index=True), Y_test)
#print(X_train+X_train)

model:xgb train AUC[0.9743] test AUC[0.9290] test ACC[0.9056]
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.2, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=4, missing=None,
       n_estimators=152, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.9)


In [104]:
def saveAns(model, X, name="ans.csv"):
    real = model.predict_proba(X)
    ans = pd.DataFrame()
    ans["ID"] = test.ID
    ans["pred"] = pd.DataFrame(real[:,1])
    ans.head()
    ans.to_csv("../" + name, index=False)
saveAns(clf, itest[code_cols + ori_cols], name="xgb_adjust_param_by_cv_and_grid.csv")

In [102]:
clf = fit_model["xgb"]
test_predict = clf.predict_proba(X_test)
train_predict = clf.predict_proba(X_train)
print("test AUC:%.4f" % metrics.roc_auc_score(Y_test, test_predict[:,1]))
print("train AUC:%.4f" % metrics.roc_auc_score(Y_train, train_predict[:,1]))
print("test Accuracy:%.4f" % metrics.accuracy_score(Y_test, clf.predict(X_test)))

test AUC:0.9669
train AUC:0.9692
test Accuracy:0.9372


# 组合模型

## stacking

In [14]:
from sklearn.model_selection import StratifiedKFold

#8折交
def KFolds(n_folds, X, Y, X_predict, Y_predict, real_X):
    print(len(Y), n_folds, len(X))
    print(n_folds)
    kfold = StratifiedKFold(n_splits=n_folds,random_state=0,shuffle=False)
    #for train, test in kfold.split(X, Y):
    #    print(np.array(X)[train])
    #return
    #skf = list(kfold.split(X, Y))
    #skf = kfold.split(X, Y)
    #skf_len = len()
    dataset_blend_train = np.zeros((X.shape[0], n_folds))
    dataset_blend_test = np.zeros((X_predict.shape[0], n_folds))
    dataset_real = np.zeros((real_X.shape[0], n_folds))
    for j, clf in enumerate(models.values()):
        dataset_blend_test_j = np.zeros((X_test.shape[0], n_folds))
        dataset_blend_real_j = np.zeros((real_X.shape[0], n_folds))
        i = 0
        for train, test in kfold.split(X, Y):
            '''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
            x_train, y_train, x_test, y_test = np.array(X)[train], np.array(Y)[train], np.array(X)[test], np.array(Y)[test]
            clf.fit(x_train, y_train)
            y_submission = clf.predict_proba(x_test)[:, 1]
            dataset_blend_train[test, j] = y_submission
            dataset_blend_test_j[:, i] = clf.predict_proba(X_predict)[:, 1]
            dataset_blend_real_j[:, i] = clf.predict_proba(real_X)[:, 1]
            i+=1
        '''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
        dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
        dataset_real[:, j] = dataset_blend_real_j.mean(1)
    clf = models["gbdt"]
    clf.fit(dataset_blend_train, Y)
    y_submission = clf.predict_proba(dataset_blend_test)[:, 1]
    real_y = clf.predict_proba(dataset_real)[:, 1]
    print("model:%s test AUC[%.4f]" % 
      ("blending", 
       metrics.roc_auc_score(Y_predict, y_submission)
      ))
    return real_y
split_len = int(len(X_train)/len(models))*len(models)
split_len = len(X_train)
print(split_len)
real_test = KFolds(len(models), X_train[:split_len], Y_train[:split_len], X_test, Y_test, itest[code_cols + ori_cols])

17721
17721 7 17721
7


ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15'] ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
expected f5, f13, f12, f9, f0, f10, f4, f2, f14, f15, f7, f6, f3, f8, f11, f1 in input data
training data did not have the following fields: campaign, housing, month, age, loan, marital, day, pdays, balance, education, previous, job, contact, duration, default, poutcome

In [ ]:
real = clf.predict_proba(itest[code_cols + ori_cols])
ans = pd.DataFrame()
ans["ID"] = test.ID
ans["pred"] = pd.DataFrame(real[:,1])
ans.head()
ans.to_csv("../ans_blend_0103.csv", index=False)

# 测试数据

In [ ]:
real = clf.predict_proba(itest[code_cols + ori_cols])
ans = pd.DataFrame()
ans["ID"] = test.ID
ans["pred"] = pd.DataFrame(real[:,1])
ans.tail()

In [ ]:
ans.to_csv("../ans_XGB_0103.csv", index=False)